In [2]:
from inference import get_model
#from google.colab import userdata

ROBOFLOW_API_KEY = "ITfUpuY5QO9WTBpcEXTh"  # Replace with your actual Roboflow API key

GOAL_DETECTION_MODEL_ID = "football-goalpost/3"
GOAL_DETECTION_MODEL = get_model(model_id=GOAL_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

[04/05/25 13:15:20] WARNING  Your inference package version 0.43.0 is out of date! Please upgrade to ]8;id=240415;file://c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\inference\core\__init__.py\__init__.py]8;;\:]8;id=231346;file://c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\inference\core\__init__.py#41\41]8;;\
                             version 0.46.0 of inference for the latest features and bug fixes by                  
                             running `pip install --upgrade inference`.                                            

ModelDependencyMissing: Your `inference` configuration does not support PaliGemma model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support Florence2 model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support Qwen2.5-VL model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[clip]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does

In [ ]:
import supervision as sv
import numpy as np  

SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/videos/goalMatch.mp4"

box_annotator = sv.BoxAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)

frame_number = 2000  # Change to the desired frame number
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

for i, frame in enumerate(frame_generator):
    if i == frame_number:
        # Run inference on the specific frame
        result = GOAL_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
        detections = sv.Detections.from_inference(result)

        label_annotator = sv.LabelAnnotator(##labeling the box and adding text
          color = sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
          text_color = sv.Color.from_hex('#000000')
           )
        # Annotate the frame
        annotated_frame = frame.copy()
        annotated_frame = box_annotator.annotate(
            scene=annotated_frame,
            detections=detections
          )

        labels = [
           f"{class_name} {confidence:.2f}"
          for class_name, confidence
          in zip(detections["class_name"] , detections.confidence)
           ]

        annotated_frame = label_annotator.annotate(annotated_frame , detections,labels =labels)##annotate the frame with the labels
 
        # Show the annotated frame
        sv.plot_image(annotated_frame)


In [ ]:
import supervision as sv
import numpy as np
import cv2

SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/videos/goalMatch.mp4"

# Box annotator setup with color palette and thickness
box_annotator = sv.BoxAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)

label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000')
)


# Frame generator from the video
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

# Define the font for text labels
font = cv2.FONT_HERSHEY_SIMPLEX

# Loop through frames in the video
for frame in frame_generator:
    # Run inference on the current frame with confidence threshold
    result = GOAL_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    
    # Convert the inference result to detections
    detections = sv.Detections.from_inference(result)

    # Copy the current frame for annotation
    annotated_frame = frame.copy()
    
    # Annotate the frame with bounding boxes
    annotated_frame = box_annotator.annotate(
        scene=annotated_frame,
        detections=detections
    )
    
    labels = [
          f"{class_name} {confidence:.2f}"
          for class_name, confidence
          in zip(detections["class_name"] , detections.confidence)
         ]
    annotated_frame = label_annotator.annotate(annotated_frame , detections,labels =labels)##annotate the frame with the labels

    # Show the annotated frame
    sv.plot_image(annotated_frame)
    break;


In [ ]:
import cv2
import numpy as np

# Load YOLO model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")  # Make sure to replace with correct paths
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load class labels (COCO dataset, for example)
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load image (or use video frame)
image = cv2.imread("image.jpg")  # Replace with your image path
height, width, channels = image.shape

# Prepare the image for YOLO
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)

# Get the output from the YOLO network
outs = net.forward(output_layers)

# Initialize lists to hold the coordinates of detected objects
coordinates = []

# Loop over each output of the YOLO model
for out in outs:
    for detection in out:
        scores = detection[5:]  # The class scores
        class_id = np.argmax(scores)  # Get the class ID of the object
        confidence = scores[class_id]  # Get the confidence of the prediction

        # Only consider detections with high confidence
        if confidence > 0.5:  # Confidence threshold (can be adjusted)
            # Get the coordinates of the bounding box
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Get top-left corner of the bounding box
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            # Store the coordinates (x, y, w, h)
            coordinates.append((x, y, w, h))

# Print the coordinates of detected objects
print("Detected Coordinates (x, y, w, h):")
for coord in coordinates:
    print(coord)

# Optionally, draw the bounding boxes on the image
for coord in coordinates:
    x, y, w, h = coord
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Show the result
cv2.imshow("Image with Detected Objects", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/videos/goalMatch.mp4"

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)  # Get the first frame from the generator
image = frame.copy()  # Copy the frame for processing

# Resize the image to match the model's expected input dimensions (e.g., 640x640)
image_resized = cv2.resize(image, (640, 640))

# Normalize the image and convert to float32
image_resized = image_resized.astype('float32') / 255.0

# Reorder dimensions to match the model's expected input format (channels-first)
image_resized = np.transpose(image_resized, (2, 0, 1))  # Convert from (640, 640, 3) to (3, 640, 640)

# Add a batch dimension to the image
image_resized = np.expand_dims(image_resized, axis=0)  # Convert to (1, 3, 640, 640)

# Perform inference to get predictions
# Perform inference to get predictions
predictions = GOAL_DETECTION_MODEL.predict(image_resized, confidence=40, overlap=30)

predicted_bboxes = []  # List to store predicted bounding boxes

# Check the structure of predictions and extract bounding box and class details
for prediction in predictions[0]:  # Access the first element of the predictions tuple
    x, y, w, h = prediction[0], prediction[1], prediction[2], prediction[3]  # Adjust indices based on the actual structure
    confidence = prediction[4]  # Assuming confidence is at index 4
    class_id = prediction[5]  # Assuming class_id is at index 5
    predicted_bboxes.append({
        "class_id": class_id,
        "confidence": confidence,
        "x": x,
        "y": y,
        "w": w,
        "h": h
    })

print("Saved Predictions:")
for bbox in predicted_bboxes:
    print(bbox)

In [ ]:
import supervision as sv
import cv2
import numpy as np

SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/videos/goalMatch.mp4"

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)  # Get the first frame from the generator
image_path = "C:/Users/user/Desktop/FootballAI/videos/goal1.png"  # Path to the image
image = cv2.imread(image_path)  # Load the image using OpenCV

result = GOAL_DETECTION_MODEL.infer(image, confidence=0.3)[0]
detections = sv.Detections.from_inference(result)

# Resize the image to match the model's expected input dimensions (e.g., 1280x1280)
image_resized = cv2.resize(image, (1280, 1280))

# Normalize the image and convert to float32
image_resized = image_resized.astype('float32') / 255.0

# Reorder dimensions to match the model's expected input format (channels-first)
image_resized = np.transpose(image_resized, (2, 0, 1))  # Convert from (640, 640, 3) to (3, 640, 640)

# Add a batch dimension to the image
image_resized = np.expand_dims(image_resized, axis=0)  # Convert to (1, 3, 640, 640)

# Perform inference to get predictions
# Ensure the image has the correct shape (height, width, channels)
image_resized_corrected = np.transpose(image_resized[0], (1, 2, 0))  # Convert from (1, 3, 1280, 1280) to (1280, 1280, 3)

# Perform inference to get predictions
predictions = GOAL_DETECTION_MODEL.infer(image_resized_corrected, confidence=0.4)[0]  # Use the correct method and confidence threshold

predicted_bboxes = []  # List to store predicted bounding boxes
BALL_ID = 0  # Assuming class ID for ball is 0
# Check the structure of predictions and extract bounding box and class details
for detection, class_id in zip(detections.xyxy, detections.class_id):
    if class_id == BALL_ID:  # Check if the detection corresponds to the ball
        x_min, y_min, x_max, y_max = detection
        ball_coords = ((x_min + x_max) / 2, (y_min + y_max) / 2)  # Get the center of the bounding box for the ball
        print(f"GOALPOST detected: {ball_coords}")


In [ ]:
import supervision as sv
import cv2
import numpy as np

SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/videos/goalMatch.mp4"

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)  # Get the first frame from the generator
image_path = "C:/Users/user/Desktop/FootballAI/videos/goal1.png"  # Path to the image
image = cv2.imread(image_path)  # Load the image using OpenCV

# Resize the image to match the model's expected input dimensions (e.g., 1280x1280)
image_resized = cv2.resize(image, (1280, 1280))

# Normalize the image to [0, 1] range (if necessary)
image_resized = image_resized.astype('float32') / 255.0

# Reorder dimensions to match the model's expected input format (channels-first)
image_resized = np.transpose(image_resized, (2, 0, 1))  # Convert from (1280, 1280, 3) to (3, 1280, 1280)

# Add a batch dimension to the image
image_resized = np.expand_dims(image_resized, axis=0)  # Convert to (1, 3, 1280, 1280)

# Perform inference again with preprocessed image (remove batch dimension)
image_resized_corrected = np.transpose(image_resized[0], (1, 2, 0))  # Convert from (1, 3, 1280, 1280) to (1280, 1280, 3)
result = GOAL_DETECTION_MODEL.infer(image_resized_corrected, confidence=0.2)[0]
detections = sv.Detections.from_inference(result)
print("Detections:", detections)

# Perform inference to get predictions
result = GOAL_DETECTION_MODEL.infer(image, confidence=0.3)[0]
detections = sv.Detections.from_inference(result)

# Print the inference result to check its structure
print("Inference result:", result)

# Print the detections object to check the bounding boxes and class ids
print("Detections:", detections)

# Extract bounding boxes and class IDs
predicted_bboxes = []  # List to store predicted bounding boxes
BALL_ID = 0  # Assuming class ID for ball is 0

# Check the structure of predictions and extract bounding box and class details
for detection, class_id in zip(detections.xyxy, detections.class_id):
    print(f"Detection: {detection}, Class ID: {class_id}")  # Debugging output

    if class_id == BALL_ID:  # Check if the detection corresponds to the ball
        x_min, y_min, x_max, y_max = detection
        ball_coords = ((x_min + x_max) / 2, (y_min + y_max) / 2)  # Get the center of the bounding box for the ball
        print(f"Ball detected: {ball_coords}")

# If no ball was detected, print a message
if not predicted_bboxes:
    print("No ball detected.")


In [ ]:
from math import sqrt
# Function to calculate Euclidean distance
def calculate_distance(ball_center, goalpost_center):
    return sqrt((goalpost_center[0] - ball_center[0])**2 + (goalpost_center[1] - ball_center[1])**2)

In [13]:
goalpost_center = None  # Variable to store the center of the goalpost

image_path = "C:/Users/user/Desktop/FootballAI/videos/image.png"  # Path to the image
image = cv2.imread(image_path)  # Load the image using OpenCV

result = GOAL_DETECTION_MODEL.infer(image, confidence=0.3)[0]

detections = sv.Detections.from_inference(result)
print(detections.xyxy)

for detection, class_id in zip(detections.xyxy, detections.class_id):
        x_min, y_min, x_max, y_max = detection
        goalpost_center = ((x_min + x_max) / 2, (y_min + y_max) / 2)  # Calculate the center of the goalpost
        print(f"Goalpost center: {goalpost_center}")  # Print the center coordinates
        break  # Stop after finding the first goalpost


[[116. 203. 212. 308.]]
Goalpost center: (164.0, 255.5)


In [4]:
import cv2
import numpy as np
import supervision as sv
goalpost_center = None  # Variable to store the center of the goalpost

image_path = "C:/Users/user/Desktop/FootballAI/videos/image.png"  # Path to the image
image = cv2.imread(image_path)  # Load the image using OpenCV

result = GOAL_DETECTION_MODEL.infer(image, confidence=0.3)[0]

detections = sv.Detections.from_inference(result)
print(detections.class_id)

for detection, class_id in zip(detections.xyxy, detections.class_id):
    x_min, y_min, x_max, y_max = detection
    goalpost_center = ((x_min + x_max) / 2, (y_min + y_max) / 2)  # Calculate the center of the goalpost
    print(f"Goalpost center: {goalpost_center}")  # Print the center coordinates
    
    # Draw a red dotted circle at the center
    center_coordinates = (int(goalpost_center[0]), int(goalpost_center[1]))
    radius = 5
    color = (0, 0, 255)  # Red color in BGR
    thickness = 2
    line_type = cv2.LINE_AA  # Anti-aliased line for better quality
    cv2.circle(image, center_coordinates, radius, color, thickness, lineType=line_type)
    
    break  # Stop after finding the first goalpost

# Display the image with the red dotted circle
cv2.imshow("Image with Center Point", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


[0]
Goalpost center: (164.0, 255.5)
